In [3]:
import os
import pandas as pd
from lxml import etree
import datetime
from datetime import datetime
import time


ns = {'tei': 'http://www.tei-c.org/ns/1.0', "xml": "http://www.w3.org/XML/1998/namespace"}



def readfiles(pathtofiles):
    eventslist = []
    for filename in os.listdir(pathtofiles):
        regesttree = etree.parse(pathtofiles+"/"+filename)
        regestroot = regesttree.getroot()
        creationdate = regestroot.find(".//tei:creation/tei:date", ns)
        origindate = regestroot.find(".//tei:history/tei:origin", ns)
        if creationdate is not None:
           date = createdate(creationdate)
        elif origindate is not None:
           date = createdate(origindate)
        else:
           date = " "
        eventNodes = regestroot.xpath(".//*[@type='event'][./tei:catchwords/@n='disp']", namespaces=ns)
        if len(eventNodes) > 0:
            print("--------------------------")
            print('filename: ',filename)
            print('datum: ',date)
            for eventNode in eventNodes:
                print('event: ' ,eventNode.attrib['ref'])
                eventsDict = {}
                eventsDict['xml'] = filename
                eventsDict['date'] = date
                catchwordslist =[]
                eventsDict['event_ref'] = eventNode.attrib['ref']
                catchwords = eventNode.xpath(".//tei:catchwords[@n='disp']", namespaces=ns)
                for catchword in catchwords:
                 #   print('catchword: ',catchword.xpath("normalize-space(.//text())"))
                    catchwordslist.append(catchword.xpath("normalize-space(.//text())"))
                catchwordstring = " | ".join(catchwordslist)
                print("catchwords: ", catchwordstring)
                eventsDict['catchwords'] = catchwordstring
                eventslist.append(eventsDict)
    createcoloumns(eventslist)

def createcoloumns(eventslist):
    listoferrors ={}

    catchwordsFrame = pd.DataFrame()
    xmls = []
    dates = []
    events =[]
    catchwords =[]

    for event in eventslist:
        eventerror = []
        xmls.append(event['xml'])
        try:
            dates.append(event['date'][0])
        except:
            dates.append(' ')
            eventerror.append('Datum Fehlerhaft')

        try:
            events.append(event['event_ref'])
        except:
            print(event)
            events.append(' ')
            eventerror.append('Eventref Fehlerhaft')

        try:
            catchwords.append(event['catchwords'])
        except:
            catchwords.append(' ')
            eventerror.append('Catchwords fehlerhaft')

        if len(eventerror) > 0:
            listoferrors[event['xml']] = eventerror

    writeerrorfile(listoferrors)

    catchwordsFrame['xml'] = xmls
    catchwordsFrame['date'] = dates
    catchwordsFrame['event'] = events
    catchwordsFrame['catchwords'] = catchwords

    catchwordsFrame.to_csv("../data/Catchwords.csv", sep="\t", encoding="utf-8")


def writeerrorfile(listoferrors):
    with open("../Errors/catchwords_ERROR.txt", "w") as file:

        if len(listoferrors) >0:
            file.write(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
            file.write("\n")
            for xml in listoferrors.keys():
                file.write("--------------\n")
                file.write("file: " + xml+"\n")
                for error in listoferrors[xml]:
                    file.write(error+"\n")

        else:
            file.write(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
            file.write("\nKeine ERRORS gefunden")


def createdate(node):
    if 'when' in node.attrib:
        when = node.attrib['when'].replace('-','')
        date = datetime.strptime(when, '%Y%m%d').date()
        year = str(date).split("-")[0]
        return [date,year]

    elif 'notAfter' in node.attrib:
        notafter = node.attrib['notAfter']
        if "-" in notafter:
            date = datetime.strptime(notafter, '%Y-%m-%d').date()
        else:
            date = datetime.strptime(notafter, '%Y').date()
        year = str(date).split("-")[0]
        return [date, year]
    elif 'from' in node.attrib:
        fr =node.attrib['from'].replace('-','')
        to = node.attrib['to'].replace('-','')
        fromdate = datetime.strptime(fr, '%Y%m%d').date()
        todate = datetime.strptime(to, '%Y%m%d').date()
        year = str(fromdate).split("-")[0]
        if fromdate == todate:
            return[fromdate,year]
        else:
            return [fromdate, todate,year]

readfiles("../../sources/Satzbuch_CD/1448/done")
print("done")

--------------------------
filename:  00054-eintrag_vom_1448-06-10.xml
datum:  [datetime.date(1448, 6, 10), '1448']
event:  ev__SB_CD_00054
catchwords:  habent versaczt | jerlich dint | zu purkrecht
event:  ev__haus_huehnergesslein_1_burgrecht_1
catchwords:  jerlich dint | zu purkrecht
--------------------------
filename:  00055-eintrag_vom_1448-06-21.xml
datum:  [datetime.date(1448, 6, 21), '1448']
event:  ev__SB_CD_00055
catchwords:  hat versaczt | in kauffweis an sey komen sind
event:  ev__haus_alter_fleischmarkt_1_verkauf_1
catchwords:  in kauffweis an sey komen sind
--------------------------
filename:  00056-eintrag_vom_1448-08-09.xml
datum:  [datetime.date(1448, 8, 9), '1448']
event:  ev__SB_CD_00056
catchwords:  hat versaczt die ubertewrung
--------------------------
filename:  00156-eintrag_vom_1448-12-13.xml
datum:  [datetime.date(1448, 12, 13), '1448']
event:  ev__SB_CD_00156
catchwords:  hat versaczt
event:  ev__SB_CD_00156
catchwords:  ledig gesagt
------------------------